In [1]:
import fastrepl.repl as fastrepl

fastrepl.LLMCache.disable()
fastrepl.DEBUG(0)

In [2]:
from IPython.display import clear_output

In [3]:
from datasets import load_dataset

# We have dataset for meta-eval
dataset = load_dataset("yelp_review_full", split="test")
dataset = dataset.shuffle(seed=8)
dataset = dataset.select(range(10))
dataset = dataset.rename_column("text", "input")
dataset = dataset.map(
    lambda row: {"reference": row["label"] + 1, "input": row["input"]},
    remove_columns=["label"],
)

In [4]:
labels = {
    "FIVE_STARS": "given review is likely to be 5 stars",
    "FOUR_STARS": "given review is likely to be 4 stars",
    "THREE_STARS": "given review is likely to be 3 stars",
    "TWO_STARS": "given review is likely to be 2 stars",
    "ONE_STAR": "given review is likely to be 1 star",
}

human_eval = fastrepl.HumanClassifierRich(labels=labels)


def label_to_score(example):
    label = example["prediction"]
    if label is None:
        label = human_eval.compute(example["input"])

    example["prediction"] = {
        "FIVE_STARS": 5,
        "FOUR_STARS": 4,
        "THREE_STARS": 3,
        "TWO_STARS": 2,
        "ONE_STAR": 1,
    }[label]

    return example

In [5]:
def print_metric(metric_name, predictions, references):
    metric = fastrepl.load_metric(metric_name)
    result = metric.compute(predictions=predictions, references=references)
    print(f"{metric_name}: {result[metric_name]}")

In [6]:
clear_output(wait=True)

eval = fastrepl.Evaluator(
    pipeline=[
        fastrepl.LLMChainOfThought(
            model="gpt-3.5-turbo",
            labels=labels,
            context="You will get a input text from Yelp review. Please rate it from 1 to 5 stars.",
        ),
        fastrepl.LLMClassifier(
            model="gpt-3.5-turbo",
            labels=labels,
            position_debias_strategy="consensus",
        ),
    ]
)

result2 = fastrepl.LocalRunner(evaluator=eval, dataset=dataset).run()
result2 = result2.map(label_to_score)

print_metric("accuracy", result2["prediction"], result2["reference"])
print_metric("mse", result2["prediction"], result2["reference"])
print_metric("mae", result2["prediction"], result2["reference"])

result2.to_pandas()

Output()

/Users/yujonglee/dev/fastrepl/fastrepl/fastrepl/llm.py:133: UserWarning: {model} completion truncated due to length
  warnings.warn("{model} completion truncated due to length")

2023-08-31 10:36:52,437 - 11375226880 - _common.py-_common:105 - INFO: Backing off completion(...) for 1.4s (fastrepl.llm.RetryConstantException)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Classify the following sample:

We tried out the lunch specials and found them to be pretty good.  For about $6 we both got healthy portions of 
spicy food along with fairly good service.  Take a chance and go for the hotter dishes.  :-)

 [FIVE_STARS/ONE_STAR/THREE_STARS/FOUR_STARS/TWO_STARS]:

Classify the following sample:

Whew, $10 for a whiskey ginger ale and $18 for a martini.  We must be in Las Vegas. \n\nOh wait, yes we are.  Red 
Square is worth it for the ambiance, if you ask me.  I'd like to have seen more of what they had to offer.  The 
\"ice bar\" that gets a lot of hype isn't as impressive as it sounds.\n\nThe staff were pretty slow - I think 3 of 
them behind the bar, and we waited 10 minutes before being served and once our orders were taken they were in no 
rush to serve either.  Tipped minimally, but otherwise they weren't jerks or anything.  It's not like they were 
busy, either - it was a Tuesday night in February.  \n\nI'm being kind of hard on them, but we did have fun and 
it's a neat theme.  We did not go back in the vodka room, though.  Maybe next time.

 [FOUR_STARS/ONE_STAR/THREE_STARS/FIVE_STARS/TWO_STARS]:

accuracy: 0.8
mse: 0.2
mae: 0.2


,input,reference,prediction
0,Stayed at the Wm Penn down the street for a we...,5,5
1,We tried out the lunch specials and found them...,4,5
2,Should have known better than to eat in a plac...,1,1
3,This place has sure changed...and not for the ...,2,2
4,I've been bringing my son to the owner Michael...,5,5
5,The trip to the location takes two busses and ...,1,1
6,This restaurant was suggested to me by a frien...,4,4
7,This place is unique because you are sitting o...,3,3
8,"Whew, $10 for a whiskey ginger ale and $18 for...",3,3
9,"Loove me some Pei Wei! Boyfriend isn't a fan, ...",4,5
